In [2]:
# StraddleStrategy 实例运行示例 - 真实集成版本
import sys
import os
from datetime import datetime, timedelta
from typing import Dict, Optional

# 添加vnpy路径
sys.path.append(r'e:\document\Code\vnpy\.vnpy_env\Lib\site-packages')

# 导入必要的模块
from vnpy.event import EventEngine
from vnpy.trader.object import TickData, OrderData, TradeData, ContractData
from vnpy.trader.constant import Direction, Offset, Exchange, Product, OptionType
from vnpy_optionmaster.base import OptionData, UnderlyingData, ChainData, PortfolioData
from vnpy_optionmaster.straddle_strategy import StraddleStrategy, StraddleManager
from vnpy_optionmaster.engine import OptionAlgoEngine,OptionEngine

from vnpy.trader.utility import load_json

from vnpy_scripttrader import init_cli_trading
from vnpy_ctp import CtpGateway
from vnpy_scripttrader.engine import ScriptEngine

print("模块导入成功")

模块导入成功


In [ ]:
# 连接到服务器
setting = load_json("connect_ctp.json")
engine: ScriptEngine = init_cli_trading([CtpGateway])
engine.connect_gateway(setting, "CTP")
option_engine: OptionEngine = OptionEngine(engine.main_engine, engine.event_engine)
option_algo_engine: OptionAlgoEngine = OptionAlgoEngine(option_engine)

portfolios = PortfolioData("test_portfolios",  engine.event_engine)

portfolios.add_option()

option_engine.portfolios


In [4]:
# 创建事件引擎和组合管理
event_engine = EventEngine()
portfolio_name = "IO.CFFEX"  # 中金所股指期权组合

# 创建投资组合
portfolio = PortfolioData(portfolio_name, event_engine)

# 创建算法引擎 (简化版本，实际使用中需要完整的主引擎)
class MockMainEngine:
    """简化的主引擎用于演示"""
    def __init__(self):
        self.event_engine = event_engine
        
    def send_order(self, *args, **kwargs):
        return "mock_order_id"
        
    def get_contract(self, vt_symbol: str):
        return None

main_engine = MockMainEngine()
algo_engine = OptionAlgoEngine(main_engine, event_engine)
algo_engine.portfolios[portfolio_name] = portfolio

print("引擎和组合创建成功")
print(f"组合名称: {portfolio_name}")

TypeError: OptionAlgoEngine.__init__() takes 2 positional arguments but 3 were given

In [ ]:
# 创建真实的合约数据
def create_option_contract(symbol: str, strike: float, option_type: OptionType, expiry: datetime) -> ContractData:
    """创建期权合约"""
    contract = ContractData(
        symbol=symbol,
        exchange=Exchange.CFFEX,
        name=f"IO期权 {strike}",
        product=Product.OPTION,
        size=300,  # 中金所股指期权合约乘数
        pricetick=0.2,
        min_volume=1,
        option_strike=strike,
        option_expiry=expiry,
        option_type=option_type,
        option_underlying="IF",
        option_index=str(int(strike)),
        gateway_name="ctp"
    )
    return contract

def create_underlying_contract() -> ContractData:
    """创建标的合约"""
    contract = ContractData(
        symbol="IF2412",
        exchange=Exchange.CFFEX,
        name="沪深300股指期货",
        product=Product.FUTURES,
        size=300,
        pricetick=0.2,
        min_volume=1,
        gateway_name="ctp"
    )
    return contract

# 创建标的合约和期权合约
expiry_date = datetime(2024, 12, 20)  # 12月到期
underlying_contract = create_underlying_contract()

# 创建多个执行价格的期权合约
strikes = [3000, 3050, 3100, 3150, 3200]
call_contracts = {}
put_contracts = {}

for strike in strikes:
    # 看涨期权
    call_symbol = f"IO2412-C-{strike}"
    call_contracts[str(strike)] = create_option_contract(
        call_symbol, strike, OptionType.CALL, expiry_date
    )
    
    # 看跌期权  
    put_symbol = f"IO2412-P-{strike}"
    put_contracts[str(strike)] = create_option_contract(
        put_symbol, strike, OptionType.PUT, expiry_date
    )

print("合约数据创建完成")
print(f"标的合约: {underlying_contract.vt_symbol}")
print(f"期权执行价格: {strikes}")

策略参数配置:
  target_volume: 2
  max_loss: 5000.0
  profit_target: 2000.0
  days_to_close: 5
  delta_limit: 100.0
  gamma_limit: 50.0
  vega_limit: 100.0
  hedge_active: True
  hedge_delta_threshold: 50.0


In [ ]:
# 将合约添加到投资组合
print("\n=== 创建投资组合和期权链 ===")

# 添加期权合约到组合
for strike in strikes:
    strike_str = str(strike)
    portfolio.add_option(call_contracts[strike_str])
    portfolio.add_option(put_contracts[strike_str])

print(f"添加了 {len(strikes)} 个执行价格的期权对")

# 设置链的标的
chain_symbol = f"IO.{Exchange.CFFEX.value}"
portfolio.set_chain_underlying(chain_symbol, underlying_contract)

# 获取期权链
chain = portfolio.chains[chain_symbol]
print(f"期权链创建成功: {chain_symbol}")
print(f"链中期权数量: {len(chain.options)}")
print(f"可用执行价格: {chain.indexes}")

# 设置利率和定价模型
portfolio.set_interest_rate(0.03)  # 3%年利率

# 简化的定价模型（实际应用中使用Black-Scholes等）
class MockPricingModel:
    @staticmethod
    def calculate_price(*args):
        return 0.1  # 简化的理论价格
    
    @staticmethod 
    def calculate_greeks(*args):
        return 0.1, 0.5, 0.01, 0.02, 0.1  # price, delta, gamma, theta, vega
    
    @staticmethod
    def calculate_impv(*args):
        return 0.2  # 简化的隐含波动率

portfolio.set_pricing_model(MockPricingModel)
print("定价模型设置完成")


=== 启动跨式策略 ===
发送委托: 10004186.CFFEX 空 开 2@0.153
11:05:21 [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.SHORT 2@0.153
Engine Log: [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.SHORT 2@0.153
11:05:21 [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.SHORT Offset.OPEN 2@0.153 [mock_order_1]
Engine Log: [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.SHORT Offset.OPEN 2@0.153 [mock_order_1]
发送委托: 10004187.CFFEX 空 开 2@0.143
11:05:21 [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.SHORT 2@0.143
Engine Log: [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.SHORT 2@0.143
11:05:21 [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.SHORT Offset.OPEN 2@0.143 [mock_order_2]
Engine Log: [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.SHORT Offset.OPEN 2@0.143 [mock_order_2]
11:05:21 [ETF_Straddle_3000] 跨式策略启动：10004186.CFFEX & 10004187.CFFEX
Engine Log: [ETF_Straddle_3000] 跨式策略启动：10004186.CFFEX & 10004187.CFFEX
策略启动成功!

策略状态:
  strategy_name: ETF_Straddle_3000
  active: True
  strike_i

In [ ]:
# 创建真实的行情数据
def create_real_tick(vt_symbol: str, last_price: float, bid: float, ask: float) -> TickData:
    """创建真实的行情数据"""
    symbol, exchange = vt_symbol.split('.')
    tick = TickData(
        symbol=symbol,
        exchange=Exchange(exchange),
        datetime=datetime.now(),
        last_price=last_price,
        bid_price_1=bid,
        ask_price_1=ask,
        bid_volume_1=10,
        ask_volume_1=10,
        gateway_name="ctp"
    )
    return tick

# 为标的和期权创建行情
print("\n=== 创建行情数据 ===")

# 标的行情 (IF2412 = 3100点)
underlying_tick = create_real_tick("IF2412.CFFEX", 3100.0, 3099.8, 3100.2)
underlying = chain.underlying
underlying.update_tick(underlying_tick)

print(f"标的价格: {underlying.mid_price}")

# 期权行情 (根据实值虚值程度设置不同价格)
option_prices = {
    "3000": {"call": 120.0, "put": 20.0},   # 实值看涨，虚值看跌
    "3050": {"call": 80.0, "put": 30.0},
    "3100": {"call": 50.0, "put": 50.0},    # 平值
    "3150": {"call": 30.0, "put": 80.0},
    "3200": {"call": 20.0, "put": 120.0}    # 虚值看涨，实值看跌
}

for strike_str, prices in option_prices.items():
    # 更新看涨期权行情
    call_option = chain.calls[strike_str]
    call_tick = create_real_tick(
        call_option.vt_symbol, 
        prices["call"], 
        prices["call"] - 0.2, 
        prices["call"] + 0.2
    )
    call_option.update_tick(call_tick)
    
    # 更新看跌期权行情  
    put_option = chain.puts[strike_str]
    put_tick = create_real_tick(
        put_option.vt_symbol,
        prices["put"],
        prices["put"] - 0.2,
        prices["put"] + 0.2
    )
    put_option.update_tick(put_tick)

print("期权行情数据创建完成")

# 选择平值期权进行跨式交易
target_strike = "3100"
print(f"选择执行价格 {target_strike} 进行跨式交易")


=== 模拟市场行情变化 ===

场景1: 市场平稳，期权价格下跌
当前盈亏: 1220.00 元
看涨期权价格: 0.12 (入场价格: 0.153)
看跌期权价格: 0.115 (入场价格: 0.143)


In [ ]:
# 创建并配置跨式策略
print("\n=== 创建跨式策略 ===")

strategy = StraddleStrategy(
    strategy_name="IO_Straddle_3100",
    algo_engine=algo_engine,
    chain=chain,
    strike_index=target_strike
)

# 策略参数配置
strategy_params = {
    "target_volume": 1,           # 目标持仓1手  
    "max_loss": 15000.0,         # 最大亏损15000元 (50点 * 300倍数)
    "profit_target": 6000.0,     # 盈利目标6000元
    "days_to_close": 5,          # 距离到期5天平仓
    "delta_limit": 150.0,        # Delta限制
    "gamma_limit": 50.0,         # Gamma限制  
    "vega_limit": 100.0,         # Vega限制
    "hedge_active": True,        # 启用动态对冲
    "hedge_delta_threshold": 100.0 # Delta对冲阈值
}

print("策略参数配置:")
for key, value in strategy_params.items():
    print(f"  {key}: {value}")

# 启动策略
print("\n=== 启动跨式策略 ===")
success = strategy.start_strategy(strategy_params)

if success:
    print("策略启动成功!")
    
    # 显示策略状态
    status = strategy.get_status()
    print("\n策略状态:")
    for key, value in status.items():
        print(f"  {key}: {value}")
        
    # 显示期权基本信息
    call_opt = strategy.call_option
    put_opt = strategy.put_option
    print(f"\n期权详情:")
    print(f"看涨期权: {call_opt.vt_symbol}, 当前价格: {call_opt.mid_price}")
    print(f"看跌期权: {put_opt.vt_symbol}, 当前价格: {put_opt.mid_price}")
    print(f"距离到期: {call_opt.days_to_expiry} 天")
else:
    print("策略启动失败!")


场景2: 市场大幅波动，期权价格上涨
当前盈亏: -1680.00 元
看涨期权价格: 0.2
看跌期权价格: 0.18


In [ ]:
# 模拟交易执行和市场变化
print("\n=== 模拟交易和市场变化 ===")

# 模拟策略已经开仓（卖出跨式）
call_option = strategy.call_option
put_option = strategy.put_option

# 设置入场价格（记录卖出价格）
strategy.entry_call_price = 50.0  # 卖出看涨期权价格
strategy.entry_put_price = 50.0   # 卖出看跌期权价格
strategy.total_premium = (50.0 + 50.0) * 1 * 300  # 总收取权利金

# 模拟持仓（已卖出）
call_option.net_pos = -1
put_option.net_pos = -1

print(f"入场价格 - 看涨: {strategy.entry_call_price}, 看跌: {strategy.entry_put_price}")
print(f"总收取权利金: {strategy.total_premium} 元")

# 场景1: 市场小幅波动，期权价值衰减（有利于卖方）
print("\n场景1: 市场小幅波动，时间价值衰减")
new_call_tick = create_real_tick(call_option.vt_symbol, 45.0, 44.8, 45.2)
new_put_tick = create_real_tick(put_option.vt_symbol, 45.0, 44.8, 45.2)

call_option.update_tick(new_call_tick)
put_option.update_tick(new_put_tick)

# 触发策略行情更新
strategy.on_tick(new_call_tick)
strategy.on_tick(new_put_tick)

current_pnl = strategy.calculate_pnl()
print(f"当前盈亏: {current_pnl:.2f} 元")
print(f"看涨期权价格: {call_option.tick.last_price} (入场: {strategy.entry_call_price})")
print(f"看跌期权价格: {put_option.tick.last_price} (入场: {strategy.entry_put_price})")


场景3: 临近到期，触发时间止损
11:06:06 [ETF_Straddle_3000] 时间止损：距离到期日过近，开始平仓
Engine Log: [ETF_Straddle_3000] 时间止损：距离到期日过近，开始平仓
发送委托: 10004186.CFFEX 多 平 2@0.202
11:06:06 [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.LONG 2@0.202
Engine Log: [ETF_Straddle_3000] 策略成交：10004186.CFFEX Direction.LONG 2@0.202
11:06:06 [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.LONG Offset.CLOSE 2@0.202 [mock_order_3]
Engine Log: [ETF_Straddle_3000] 发送委托：10004186.CFFEX Direction.LONG Offset.CLOSE 2@0.202 [mock_order_3]
发送委托: 10004187.CFFEX 多 平 2@0.182
11:06:06 [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.LONG 2@0.182
Engine Log: [ETF_Straddle_3000] 策略成交：10004187.CFFEX Direction.LONG 2@0.182
11:06:06 [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.LONG Offset.CLOSE 2@0.182 [mock_order_4]
Engine Log: [ETF_Straddle_3000] 发送委托：10004187.CFFEX Direction.LONG Offset.CLOSE 2@0.182 [mock_order_4]
距离到期天数: 3
时间止损设置: 5 天


In [ ]:
# 场景2: 市场大幅波动，期权价格上涨
print("\n场景2: 市场大幅波动，期权价格上涨")

# 假设标的大涨到3200
underlying_tick_up = create_real_tick("IF2412.CFFEX", 3200.0, 3199.8, 3200.2)
underlying.update_tick(underlying_tick_up)

# 期权价格相应变化
high_call_tick = create_real_tick(call_option.vt_symbol, 120.0, 119.8, 120.2)  # 看涨大涨
high_put_tick = create_real_tick(put_option.vt_symbol, 25.0, 24.8, 25.2)      # 看跌大跌

call_option.update_tick(high_call_tick)
put_option.update_tick(high_put_tick)

strategy.on_tick(high_call_tick)
strategy.on_tick(high_put_tick)

current_pnl = strategy.calculate_pnl()
print(f"标的价格变化: 3100 -> 3200")
print(f"当前盈亏: {current_pnl:.2f} 元")
print(f"看涨期权价格: {call_option.tick.last_price}")
print(f"看跌期权价格: {put_option.tick.last_price}")

# 检查风控
if current_pnl <= -strategy.max_loss:
    print("⚠️ 触发最大亏损限制!")
elif current_pnl >= strategy.profit_target:
    print("✅ 达到盈利目标!")

# 计算总的希腊字母风险
total_delta = call_option.pos_delta + put_option.pos_delta if hasattr(call_option, 'pos_delta') else 0
print(f"当前总Delta: {total_delta:.2f}")


=== 策略管理器演示 ===
创建策略: Straddle_2950, 执行价格: 2950
创建策略: Straddle_3000, 执行价格: 3000
创建策略: Straddle_3050, 执行价格: 3050

总共创建了 3 个策略
策略 Straddle_2950: 活跃=False, 执行价格=2950
策略 Straddle_3000: 活跃=False, 执行价格=3000
策略 Straddle_3050: 活跃=False, 执行价格=3050


In [ ]:
# 使用策略管理器进行多策略管理
print("\n=== 多策略管理演示 ===")

manager = StraddleManager(algo_engine)

# 创建多个不同执行价格的跨式策略
target_strikes = ["3050", "3100", "3150"]

for strike in target_strikes:
    strategy_name = f"IO_Straddle_{strike}"
    
    try:
        multi_strategy = manager.create_strategy(strategy_name, chain, strike)
        print(f"创建策略: {strategy_name}")
        
        # 配置并启动策略
        params = strategy_params.copy()
        params["target_volume"] = 1
        
        if multi_strategy.start_strategy(params):
            print(f"  ✅ 策略 {strategy_name} 启动成功")
        else:
            print(f"  ❌ 策略 {strategy_name} 启动失败")
            
    except Exception as e:
        print(f"  ❌ 创建策略 {strategy_name} 失败: {e}")

print(f"\n策略管理器中共有 {len(manager.get_all_strategies())} 个策略")

# 模拟批量行情更新
print("\n模拟批量行情更新...")
for strategy in manager.get_all_strategies():
    # 获取该策略的期权
    if strategy.call_option and strategy.put_option:
        # 模拟行情变化
        call_tick = create_real_tick(strategy.call_option.vt_symbol, 60.0, 59.8, 60.2)
        put_tick = create_real_tick(strategy.put_option.vt_symbol, 40.0, 39.8, 40.2)
        
        # 更新到管理器
        manager.on_tick(call_tick)
        manager.on_tick(put_tick)

print("批量更新完成")

# 策略性能汇总
print("\n=== 策略性能汇总 ===")
for strategy in manager.get_all_strategies():
    status = strategy.get_status()
    print(f"策略: {status['strategy_name']}")
    print(f"  执行价格: {status['strike_index']}")
    print(f"  活跃状态: {status['active']}")
    print(f"  距离到期: {status['days_to_expiry']} 天")
    print(f"  当前盈亏: {status['current_pnl']:.2f}")
    print()

print("=== 真实集成演示完成 ===")
print("\n本演示展示了:")
print("1. 真实的vnpy_optionmaster组件集成")
print("2. 正确的事件引擎和投资组合设置") 
print("3. 真实的合约数据和行情处理")
print("4. 完整的跨式策略生命周期管理")
print("5. 多策略并行管理和监控")


=== 策略性能分析 ===

策略名称: Straddle_2950
执行价格: 2950
看涨期权: call_2950.CFFEX
看跌期权: put_2950.CFFEX
看涨持仓: 0
看跌持仓: 0
总权利金收入: 0.00
当前盈亏: 0.00
距离到期: 30 天
入场时间: 

=== 跨式策略运行示例完成 ===
该示例展示了:
1. 如何创建和配置跨式策略
2. 如何启动策略并进行交易
3. 如何模拟不同市场场景
4. 如何使用策略管理器管理多个策略
5. 如何分析策略性能
